In [1]:
from tkinter import *
from tkinter import ttk
import tkinter.font as tkFont
from tkinter import font

In [2]:
from movie import Movie
from parse import parse_imdb_data

In [3]:
import os
import pickle

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import normalize
from sklearn import svm

In [5]:
import numpy as np
from scipy.sparse import csr_matrix, vstack, hstack, diags

In [6]:
if os.path.isfile("movies.p"):
    movies = pickle.load(open("movies.p", "rb"))
else:
    movies = parse_imdb_data()
    pickle.dump(movies, open( "movies.p", "wb" ))

In [7]:
m_ = movies
#m_=list(filter(lambda m: m.year>2000, m_))
m_=list(filter(lambda m: m.num_ratings>100, m_))
#m_=list(filter(lambda m: m.mean_rating>9.3, m_))
len(m_)

106607

In [8]:
len(movies)

399184

In [9]:
def sortby(tree, col, descending, is_number=False):
    """sort tree contents when a column header is clicked on"""
    # grab values to sort
    data = [(tree.set(child, col), child) \
        for child in tree.get_children('')]
    # if the data to be sorted is numeric change to float
    #data =  change_numeric(data)
    # now sort the data in place
    if(is_number): 
        data = [(float(d[0]), d[1]) for d in data]
    data.sort(reverse=descending)
    for ix, item in enumerate(data):
        tree.move(item[1], '', ix)
    # switch the heading so it will sort in the opposite direction
    tree.heading(col, command=lambda col=col: sortby(tree, col, \
        int(not descending), is_number))

In [10]:
min_year = "2100"
min_rating_count = min_mean_rating = float('inf')
max_year = "1800"
max_rating_count = max_mean_rating = float('-inf')
for m in m_:
    min_year = min((min_year, m.year))
    max_year = max((max_year, m.year))
    min_rating_count = min((min_rating_count, m.num_ratings))
    max_rating_count = max((max_rating_count, m.num_ratings))
    min_mean_rating = min((min_mean_rating, m.mean_rating))
    max_mean_rating = max((max_mean_rating, m.mean_rating))    

In [11]:
if os.path.isfile("ratings.p"):
    ratings = pickle.load(open("ratings.p", "rb"))
else:
    ratings = {}

In [12]:
ratings = {}

In [13]:
Dk = []
Dg = []
Dp = []
Da1 = []
Da2 = []


for m in m_:
    ms = ''
    if m.keywords is not None:
        #ms = " ".join(m.keywords)
        ms = " ".join(["Keyword:"+k for k in m.keywords])
    Dk.append(ms)
    mg = ''
    if m.genres is not None:
        mg = " ".join(m.genres)
    Dg.append(mg)
    if m.plot is not None:
        #plot = m.plot.split()
        #plot = " ".join(["Plot:"+p for p in plot])
        #Dp.append(plot)
        Dp.append(m.plot)
    else:
        Dp.append("")
    if m.actors is not None:
        a = ""
        for actor in m.actors:
            a = a + " Actor:" + actor.replace(" ", "")
        Da1.append(a)
    else:
        Da1.append("")
    if m.actresses is not None:
        a = ""
        for actress in m.actresses:
            a = a + " Actress:" + actress.replace(" ", "")
        Da2.append(a)
    else:
        Da2.append("")

In [14]:
vocabulary = []

# Genre
#vect = CountVectorizer(token_pattern = r'\b\S+\b', lowercase=False, binary=True)
#vect = TfidfVectorizer(token_pattern = r'\b\S+\b', lowercase=False)
#Xg = vect.fit_transform(Dg)
#vocabulary = vect.get_feature_names()
# X = Xg

# Keywords
vect = CountVectorizer(token_pattern = r'\b\S+\b', min_df=5, lowercase=False, binary=True, dtype=np.float)
#vect = TfidfVectorizer(token_pattern = r'\b\S+\b', min_df=5)
Xk = vect.fit_transform(Dk)
Xk = normalize(Xk)
vocabulary = vocabulary + vect.get_feature_names()
#X = hstack((Xg, Xk)).tocsr()
X = Xk


# Plot
#vect = TfidfVectorizer(token_pattern = r'\b\S+\b', min_df=5, stop_words='english')
#Xp = vect.fit_transform(Dp)
#vocabulary = vocabulary + vect.get_feature_names()
#X = hstack((X, Xp)).tocsr()

# Actors
vect = CountVectorizer(token_pattern = r'\b\S+', lowercase=False, binary=True, dtype=np.float)
Xa1 = vect.fit_transform(Da1)
Xa1 = normalize(Xa1)
vocabulary = vocabulary + vect.get_feature_names()
X = hstack((X, Xa1)).tocsr()

# Actresses
vect = CountVectorizer(token_pattern = r'\b\S+', lowercase=False, binary=True, dtype=np.float)
Xa2 = vect.fit_transform(Da2)
Xa2 = normalize(Xa2)
vocabulary = vocabulary + vect.get_feature_names()
X = hstack((X, Xa2)).tocsr()

X.shape

(106607, 996843)

In [15]:
root = Tk()
root.title("IMDB MOVIE RECOMMENDER SYSTEM")

root.rowconfigure(0, weight=1)
root.columnconfigure(0, weight=1)

mainframe = ttk.Frame(root, padding="3 3 12 12")
mainframe.grid(column=0, row=0, sticky="nsew")
mainframe['borderwidth'] = 10
mainframe['relief'] = 'sunken'

mainframe.rowconfigure(0, weight=1)
mainframe.columnconfigure(0, weight=0)
mainframe.columnconfigure(1, weight=1)

filterframe = ttk.Labelframe(mainframe, padding="3 3 12 12", text="Filter")
filterframe.grid(column=0, row=0, sticky="nsew")
filterframe['borderwidth'] = 2
filterframe['relief'] = 'sunken'


dataframe = ttk.Labelframe(mainframe, padding="3 3 12 12", text="Data")
dataframe.grid(column=1, row=0, sticky="nsew")
dataframe['borderwidth'] = 2
dataframe['relief'] = 'sunken'

dataframe.grid_columnconfigure(0, weight=1)
dataframe.grid_columnconfigure(1, weight=0)
dataframe.grid_rowconfigure(0, weight=1)
dataframe.grid_rowconfigure(1, weight=0)
dataframe.grid_rowconfigure(2, weight=0)


yearfilterframe = ttk.Labelframe(filterframe, padding="3 3 12 12", text="Year")
yearfilterframe.grid(column=0, row=0, sticky="nsew")
yearfilterframe['borderwidth'] = 2
yearfilterframe['relief'] = 'sunken'
ttk.Label(yearfilterframe, text="MIN").grid(column=0, row=0, sticky="W")
minyear=StringVar()
minyear.set(min_year)
ttk.Entry(yearfilterframe, textvariable=minyear).grid(column=1, row=0, sticky="E")
ttk.Label(yearfilterframe, text="MAX").grid(column=0, row=1, sticky="W")
maxyear=StringVar()
maxyear.set(max_year)
ttk.Entry(yearfilterframe, textvariable=maxyear).grid(column=1, row=1, sticky="E")


numratingfilterframe = ttk.Labelframe(filterframe, padding="3 3 12 12", text="# of Ratings")
numratingfilterframe.grid(column=0, row=1, sticky="nsew")
numratingfilterframe['borderwidth'] = 2
numratingfilterframe['relief'] = 'sunken'
ttk.Label(numratingfilterframe, text="MIN").grid(column=0, row=0, sticky="E")
minratingcount=IntVar()
minratingcount.set(min_rating_count)
ttk.Entry(numratingfilterframe, textvariable=minratingcount).grid(column=1, row=0, sticky="W")
ttk.Label(numratingfilterframe, text="MAX").grid(column=0, row=1, sticky="E")
maxratingcount=IntVar()
maxratingcount.set(max_rating_count)
ttk.Entry(numratingfilterframe, textvariable=maxratingcount).grid(column=1, row=1, sticky="W")


meanratingfilterframe = ttk.Labelframe(filterframe, padding="3 3 12 12", text="Average Rating")
meanratingfilterframe.grid(column=0, row=2, sticky="nsew")
meanratingfilterframe['borderwidth'] = 2
meanratingfilterframe['relief'] = 'sunken'
ttk.Label(meanratingfilterframe, text="MIN").grid(column=0, row=0, sticky="W")
minrating=DoubleVar()
minrating.set(min_mean_rating)
ttk.Entry(meanratingfilterframe, textvariable=minrating).grid(column=1, row=0, sticky="E")
ttk.Label(meanratingfilterframe, text="MAX").grid(column=0, row=1, sticky="W")
maxrating=DoubleVar()
maxrating.set(max_mean_rating)
ttk.Entry(meanratingfilterframe, textvariable=maxrating).grid(column=1, row=1, sticky="E")

phrasefilterframe = ttk.Labelframe(filterframe, padding="3 3 12 12", text="Phrase")
phrasefilterframe.grid(column=0, row=3, sticky="nsew")
phrasefilterframe['borderwidth'] = 2
phrasefilterframe['relief'] = 'sunken'

titlephrase = StringVar()
ttk.Label(phrasefilterframe, text="Title").grid(column=0, row=0, sticky="W")
ttk.Entry(phrasefilterframe, textvariable=titlephrase).grid(column=1, row=0, sticky="nsew")
keywordphrase = StringVar()
ttk.Label(phrasefilterframe, text="Keyword").grid(column=0, row=1, sticky="W")
ttk.Entry(phrasefilterframe, textvariable=keywordphrase).grid(column=1, row=1, sticky="nsew")
genrephrase = StringVar()
ttk.Label(phrasefilterframe, text="Genre").grid(column=0, row=2, sticky="W")
ttk.Entry(phrasefilterframe, textvariable=genrephrase).grid(column=1, row=2, sticky="nsew")
plotphrase = StringVar()
ttk.Label(phrasefilterframe, text="Plot").grid(column=0, row=3, sticky="W")
ttk.Entry(phrasefilterframe, textvariable=plotphrase).grid(column=1, row=3, sticky="nsew")
actorphrase = StringVar()
ttk.Label(phrasefilterframe, text="Actor").grid(column=0, row=4, sticky="W")
ttk.Entry(phrasefilterframe, textvariable=actorphrase).grid(column=1, row=4, sticky="nsew")
actressphrase = StringVar()
ttk.Label(phrasefilterframe, text="Actress").grid(column=0, row=5, sticky="W")
ttk.Entry(phrasefilterframe, textvariable=actressphrase).grid(column=1, row=5, sticky="nsew")

certfilterframe = ttk.Labelframe(filterframe, padding="3 3 12 12", text="USA Certificate")
certfilterframe.grid(column=0, row=6, sticky="nsew")
certfilterframe['borderwidth'] = 2
certfilterframe['relief'] = 'sunken'


left_certificates = ["Approved", "Passed", "G", "PG", "PG-13", "R", "Unrated", "Not Rated"]
right_certificates = ["TV-G", "TV-PG", "TV-Y", "TV-Y7", "TV-14", "TV-MA"]
cert_vars = []
i=0

for cert in left_certificates:
    cv = StringVar()
    cv.set(cert)
    ttk.Checkbutton(certfilterframe, text=cert, onvalue = cert, variable=cv).grid(column=0, row=i, sticky="w")
    cert_vars.append(cv)
    i += 1

i=0

for cert in right_certificates:    
    cv = StringVar()
    cv.set(cert)
    ttk.Checkbutton(certfilterframe, text=cert, onvalue = cert, variable=cv).grid(column=2, row=i, sticky="w")
    cert_vars.append(cv)
    i += 1

cert_Unk = StringVar()
cert_Unk.set("Unknown")
ttk.Checkbutton(certfilterframe, text='Unknown', onvalue = "Unknown", variable=cert_Unk).grid(column=1, 
                                                                                            row=max((len(right_certificates),len(left_certificates)))+1, sticky="w")
cert_vars.append(cert_Unk)


removed_iids = set()

def filter_data(*args):
    remove_iids = set()
    data = data_tree.get_children('')
    #print(data_tree.set(data[0]))
    display_certificates = set()
    
    for cv in cert_vars:
        if cv.get() != "0":
            display_certificates.add(cv.get())    
    
    for iid in data:
        m = m_[int(iid)]
        # Year
        if m.year < minyear.get():
            remove_iids.add(iid)
        if m.year > maxyear.get():
            remove_iids.add(iid)
        # Rating Count
        if m.num_ratings < minratingcount.get():
            remove_iids.add(iid)
        if m.num_ratings > maxratingcount.get():
            remove_iids.add(iid)
        # MEAN RATING
        if m.mean_rating < minrating.get():
            remove_iids.add(iid)
        if m.mean_rating > maxrating.get():
            remove_iids.add(iid)
        if m.title.find(titlephrase.get()) == -1:
            remove_iids.add(iid)
        
        if genrephrase.get() != "":
            if m.genres == None:
                remove_iids.add(iid)
            else:
                gp = False
                for g in m.genres:
                    if g.find(genrephrase.get()) != -1:
                        gp = True
                        break
                if not gp:
                    remove_iids.add(iid)
        if keywordphrase.get() != "":
            if m.keywords == None:
                remove_iids.add(iid)
            else:
                kp = False
                for k in m.keywords:
                    if k.find(keywordphrase.get()) != -1:
                        kp = True
                        break
                if not kp:
                    remove_iids.add(iid)
        if plotphrase.get() != "":
            if m.plot is None:
                remove_iids.add(iid)
            else:
                m_plot_lower = m.plot.lower()
                if m_plot_lower.find(plotphrase.get().lower()) == -1:
                    remove_iids.add(iid)
        
        if actorphrase.get() != "":
            if m.actors is None:
                remove_iids.add(iid)
            else:
                actor_present = False
                for actor in m.actors:
                    if actor.lower().find(actorphrase.get().lower()) != -1:
                        actor_present = True
                        break
                if not actor_present:
                    remove_iids.add(iid)
        
        if actressphrase.get() != "":
            if m.actresses is None:
                remove_iids.add(iid)
            else:
                actress_present = False
                for actress in m.actresses:
                    if actress.lower().find(actressphrase.get().lower()) != -1:
                        actress_present = True
                        break
                if not actress_present:
                    remove_iids.add(iid)
        
                    
        if m.certificate not in display_certificates:
            remove_iids.add(iid)
        
    #data_tree.delete(list(remove_iids))
    for iid in remove_iids:
        data_tree.delete(iid)
    removed_iids.update(remove_iids)
    #TODO
    # 1. Update the other fields based on the new filtered data
    # 2. There is no way to go back in filtering right now

def reset_filters(*args):
    for iid in removed_iids:
        i = int(iid)
        item = [m_[i].title, m_[i].year, m_[i].num_ratings, m_[i].mean_rating, m_[i].certificate]
        data_tree.insert('', 'end', iid = str(i), values=item)
    #sortby(data_tree, 'Title', False)
    removed_iids.clear()
    #TODO Reset the fields to their default values
    
def double_click(e):
    item = data_tree.selection()[0]
    #print("You clicked on", data_tree.set(item))
    #print(m_[int(item)])
    tl = Toplevel(root)
    #ttk.Label(tl, text=data_tree.set(item)).grid(column=0, row=0, sticky="E")
    m = m_[int(item)]
    
    def rate_movie(*args):
        #print(m.title)
        #print(rating.get())
        #ratings[m.title+str(m.year)]=rating.get()
        if rating.get() != "no-opinion":
            ratings[m] = rating.get()
        else:
            ratings.pop(m, None)
        tl.destroy()
    
    ttk.Label(tl, text=m.title + " (" + str(m.year) + ")", font=font.Font(family='Helvetica', size=14, weight='bold')).grid(column=0, row=0, sticky="W")
    if m.genres is not None:
        g = ", ".join(m.genres)
        ttk.Label(tl, text="Genres: " + g).grid(column=0, row=1, sticky="W")      
    if m.keywords is not None:
        #cnames = StringVar(value=countrynames)
        kl= Listbox(tl, listvariable=StringVar(value=m.keywords), height=5)
        kl.grid(column=0, row=2, sticky="nsew")
        vs = ttk.Scrollbar(tl, orient=VERTICAL, command=kl.yview)
        kl.configure(yscrollcommand=vs.set)
        vs.grid(column=1, row=2, sticky="nsew")
        hs = ttk.Scrollbar(tl, orient=HORIZONTAL, command=kl.xview)
        kl.configure(xscrollcommand=hs.set)
        hs.grid(column=0, row=3, sticky="nsew")
    if m.plot is not None:
        pt = Text(tl, wrap='word')
        pt.insert('1.0', m.plot)
        pt.grid(column=0, row=4, sticky="nsew")
    if m.actors is not None:
        pt = Text(tl, wrap='word')
        pt.insert('1.0', "\n".join(m.actors))
        pt.grid(column=0, row=5, sticky="nsew")
    if m.actresses is not None:
        pt = Text(tl, wrap='word')
        pt.insert('1.0', "\n".join(m.actresses))
        pt.grid(column=0, row=6, sticky="nsew")
        
    rateframe=ttk.Frame(tl, padding="3 3 12 12")
    rateframe.grid(column=0, row=7, sticky="nsew")        
    rating = StringVar()
    if m in ratings:
        rating.set(ratings[m])
    else:
        rating.set("no-opinion")
    ttk.Radiobutton(rateframe, text='Like', variable=rating, value='like').grid(column=0, row=0, sticky="nsew")
    ttk.Radiobutton(rateframe, text='Dislike', variable=rating, value='dislike').grid(column=1, row=0, sticky="nsew")
    ttk.Radiobutton(rateframe, text='No opinion', variable=rating, value='no-opinion').grid(column=2, row=0, sticky="nsew")
    ttk.Button(rateframe, text="Submit", command=rate_movie).grid(column=1, row=1, sticky = "nsew")
        
    

buttonfilterframe = ttk.Frame(filterframe, padding="3 3 12 12")
buttonfilterframe.grid(column=0, row=5, sticky=(N, E))

ttk.Button(buttonfilterframe, text="Filter", command=filter_data).grid(column=0, row=0, sticky = "W")
ttk.Button(buttonfilterframe, text="Reset", command=reset_filters).grid(column=1, row=0, sticky = "E")

# some of the following code is borrowed from https://www.daniweb.com/programming/software-development/threads/350266/creating-table-in-python

data_header = ['Title', 'Year', 'Rating Count', "Mean Rating", 'US Certificate']
#data = [[m.title, m.year, m.num_ratings, m.mean_rating] for m in m_]

data_tree = ttk.Treeview(dataframe, columns=data_header, show="headings", selectmode='browse')

vsb = ttk.Scrollbar(dataframe, orient="vertical", command=data_tree.yview)
hsb = ttk.Scrollbar(dataframe, orient="horizontal", command=data_tree.xview)

data_tree.configure(yscrollcommand=vsb.set, xscrollcommand=hsb.set)
data_tree.grid(column=0, row=0, sticky='nsew')

data_tree.bind("<Double-1>", double_click)

vsb.grid(column=1, row=0, sticky='ns')
hsb.grid(column=0, row=1, sticky='ew')

for col in data_header:
    if col == "Rating Count" or col == "Mean Rating":
        data_tree.heading(col, text=col.title(), command=lambda c=col: sortby(data_tree, c, 0, is_number=True))
    else:
        data_tree.heading(col, text=col.title(), command=lambda c=col: sortby(data_tree, c, 0, is_number=False))
    # adjust the column's width to the header string
    data_tree.column(col, width=tkFont.Font().measure(col.title()))
    
for i in range(len(m_)):
    item = [m_[i].title, m_[i].year, m_[i].num_ratings, m_[i].mean_rating, m_[i].certificate]
    data_tree.insert('', 'end', iid = str(i), values=item)
    # adjust column's width if necessary to fit each value
    #for ix, val in enumerate(item):
     #   col_w = tkFont.Font().measure(val)
     #   if data_tree.column(data_header[ix],width=None)<col_w:
     #       data_tree.column(data_header[ix], width=col_w)

def build_a_model_of_the_data(*args):
    data = data_tree.get_children('')
    
    in_model = set()
    
    for iid in data:
        m = m_[int(iid)]
        in_model.add(m)
    y = []
    for m in m_:
        if m in in_model:
            y.append(1)
        else:
            y.append(0)
    y = np.array(y)
    clf = svm.OneClassSVM(kernel='linear')
    clf.fit(X[np.where(y==1)])
    clf_coefs = clf.coef_.toarray()[0]
    coefs_diags = diags(clf_coefs, 0)
    feat_indices = np.argsort(np.abs(clf_coefs))[::-1]
    #for i in feat_indices[:30]:
    #    print(vocabulary[i])
    #print()
    
    model_window = Toplevel(root)
    
    features_frame = ttk.Labelframe(model_window, padding="3 3 12 12", text="Features")
    features_frame.grid(column=0, row=0, sticky="nsew")
    rec_movies_frame = ttk.Labelframe(model_window, padding="3 3 12 12", text="Recommended Movies")
    rec_movies_frame.grid(column=1, row=0, sticky="nsew")
    
    model_window.columnconfigure(0, weight=1)
    model_window.columnconfigure(1, weight=1)
    model_window.rowconfigure(0, weight=1)
    
    features_frame.columnconfigure(0, weight=1)
    features_frame.rowconfigure(0, weight=1)
    
    rec_movies_frame.columnconfigure(0, weight=1)
    rec_movies_frame.rowconfigure(0, weight=1) 
    
    features_header = ['Feature', 'Weight']
    
    features_tree = ttk.Treeview(features_frame, columns=features_header, show="headings", selectmode='browse')
    features_tree.grid(column=0, row=0, sticky="nsew")    
    
    vsb = ttk.Scrollbar(features_frame, orient="vertical", command=features_tree.yview)
    features_tree.configure(yscrollcommand=vsb.set)
    vsb.grid(column=1, row=0, sticky='ns')

    
    for col in features_header:
        if col == "Weight":
            features_tree.heading(col, text=col.title(), command=lambda c=col: sortby(features_tree, c, 0, True))
        else:
            features_tree.heading(col, text=col.title(), command=lambda c=col: sortby(features_tree, c, 0, False))
        # adjust the column's width to the header string
        features_tree.column(col, width=tkFont.Font().measure(col.title()))
    
    for i in range(len(vocabulary)):
        if abs(clf_coefs[i]) > 0.01:
            item = [vocabulary[i], clf_coefs[i]]
            features_tree.insert('', 'end', iid = str(i), values=item)
    
    positive = np.where(clf.predict(X) != -1)[0]
    
    df = (clf.decision_function(X[positive])).flatten()
    
    rec_movies_header = ['Title', 'Year', 'Rating Count', "Mean Rating", 'US Certificate', 'Score']    
    
    rec_movies_tree = ttk.Treeview(rec_movies_frame, columns=rec_movies_header, show="headings", selectmode='browse')
    rec_movies_tree.grid(column=0, row=0, sticky="nsew")    
    
    vsb = ttk.Scrollbar(rec_movies_frame, orient="vertical", command=rec_movies_tree.yview)
    rec_movies_tree.configure(yscrollcommand=vsb.set)
    vsb.grid(column=1, row=0, sticky='ns')

    for col in rec_movies_header:
        if col == "Rating Count" or col == "Mean Rating" or col == "Score":
            rec_movies_tree.heading(col, text=col.title(), command=lambda c=col: sortby(rec_movies_tree, c, 0, True))
        else:
            rec_movies_tree.heading(col, text=col.title(), command=lambda c=col: sortby(rec_movies_tree, c, 0, False))
        # adjust the column's width to the header string
        rec_movies_tree.column(col, width=tkFont.Font().measure(col.title()))
    
    for i in range(len(positive)):
        m = m_[positive[i]]
        item = [m.title, m.year, m.num_ratings, m.mean_rating, m.certificate, df[i]]
        rec_movies_tree.insert('', 'end', iid = str(i), values=item)
    
    def explain_prediction(*args):
        item = rec_movies_tree.selection()[0]
        i = int(item)
        #print()
        #print(m_[positive[i]])
        #print(df[i])
        #print(clf.intercept_)
        #print(np.sum(X[positive[i]]*clf_coefs))        
        dm = X[positive[i]] * coefs_diags
        #print(dm.indices)
        #print(dm.data)
        explanation = []
        for j in range(len(dm.indices)):
            explanation.append((dm.data[j], vocabulary[dm.indices[j]]))
            #print("%s:\t%0.2f" %(vocabulary[dm.indices[j]], dm.data[j]))
        explanation = sorted(explanation)[::-1]
        
        explanation_window = Toplevel(model_window)
        
        explanation_window.columnconfigure(0, weight=1)
        explanation_window.rowconfigure(0, weight=1)
        
        explanation_header = ['Feature', 'Score']
    
        explanation_tree = ttk.Treeview(explanation_window, columns=explanation_header, show="headings", selectmode='browse')
        explanation_tree.grid(column=0, row=0, sticky="nsew")
    
        vsb = ttk.Scrollbar(explanation_window, orient="vertical", command=explanation_tree.yview)
        explanation_tree.configure(yscrollcommand=vsb.set)
        vsb.grid(column=1, row=0, sticky='ns')
        
        for col in explanation_header:
            if col == "Score":
                explanation_tree.heading(col, text=col.title(), command=lambda c=col: sortby(explanation_tree, c, 0, True))
            else:
                explanation_tree.heading(col, text=col.title(), command=lambda c=col: sortby(explanation_tree, c, 0, False))
            # adjust the column's width to the header string
            explanation_tree.column(col, width=tkFont.Font().measure(col.title()))

        for e in explanation:
            item = [e[1], e[0]]
            explanation_tree.insert('', 'end', values=item)
        
        
        #for e in explanation[:5]:
        #    print("%s:\t%0.2f" %(e[1], e[0]))
        
    
    rec_movies_tree.bind("<Double-1>", explain_prediction)
        

    

ttk.Button(dataframe, text="Build a Model of the Data", command=build_a_model_of_the_data).grid(column=0, row=2, sticky = "nsew")

root.mainloop()